# imports

In [1]:
import numpy as np

# game logic

In [51]:
# 1 is white
# 2 is black

# n: size of board (nxn), n even
def createBoard(n:int = 8):
    if n % 2 != 0: return None # n should be even
    board = np.zeros([n, n])
    board[n//2 - 1, n//2 - 1] = 1
    board[n//2, n//2 - 1] = 2
    board[n//2 - 1, n//2] = 2
    board[n//2, n//2] = 1
    return board

def printBoard(board: np.array):
    for row, rownr in zip(board,range(len(board))):
        print(rownr, end="\t")
        for val in row:
            print(int(val), end=" ")
        print()
    print("\n\t", end="")
    for colnr in range(len(board)): print(int(colnr), end=" ")
    print()    

directions = [[1, 0],
              [1, 1],
              [0, 1],
              [-1, 1],
              [-1, 0],
              [-1, -1],
              [0, -1],
              [1, -1]]
# board: current game board
# location: tuple (row, column) where user want to play
# player: 1 white player/ 2 black player
# returns: if move was succesfull, list of list of directions where it is legal
def checkLegalMove(board: np.array, location:tuple, player: int):
    #first check if move is legal
    if board.item(location) != 0: return False, []
    n = len(board)
    player2 = (player % 2) + 1
    legal = False
    legal_directions = []
    for direction in directions:
        new_x = location[0] + direction[0]
        new_y = location[1] + direction[1]
        found_player2 = False # check wether there is a player2 coin
        found_player_after_player2 = False # check wether there is an "ending" coin of player1
        while new_x >= 0 and new_x < n and new_y >= 0 and new_y < n:
            coin = board[new_x, new_y]
            if coin == 0: break
            if coin == player2: found_player2 = True
            if coin == player and found_player2:
                found_player_after_player2 = True
                legal = True
                legal_directions.append(direction)
                break
            new_x += direction[0]
            new_y += direction[1]
    return legal, legal_directions

def makeMove(board: np.array, location: tuple, legal_directions: list, player: int):
    player2 = (player % 2) + 1
    n = len(board)
    board[location[0], location[1]] = player
    for direction in legal_directions:
        new_x = location[0] + direction[0]
        new_y = location[1] + direction[1]
        while new_x >= 0 and new_x < n and new_y >= 0 and new_y < n:
            coin = board[new_x, new_y]
            if coin == player: break #equals to 0 should not happen
            if coin == player2: board[new_x, new_y] = player
            new_x += direction[0]
            new_y += direction[1]
    return True

def findAllPossibilities(board: np.array, player: int):
    possiblities = {}
    for x in range(len(board)):
        for y in range(len(board)):
            legal, legal_dir = checkLegalMove(board, (x,y), player)
            if legal: possibilities[(x,y)] = legal_dir
    return possiblities

# state given back after each call
# move wanted move when given the state
# state: dict with:
#    "board": for the board game itself
#    "cur_player": for the player that should play next
#    "possibilities": for the possibilites the player can do
#    "winner": 0 if game is ongoing, 1 if winner is white, 2 if winner is black, 3 if it is a draw
def PlayGame(state:dict = None, move:tuple = None):
    if state == None:
        board = creatBoard()
        cur_player = 1
        state = {"board": board, "cur_player": cur_player}
        
        possibilities = findAllPossibilities(board, cur_player)
        state["possibilities"] =  possibilities
        state["winner"] = 0
        return state
    #if state is given
    if move == None: return state # if no move is given
    possibilities = state["possibilities"]
    if move not in possibilities: return state #if move is not legal
    makeMove(board, move, possiblities[move], player)
    
    #look for player2 moves to play next
    player2 = (player % 2) + 1
    player2Possiblities = findAllPossibilities(state["board"], player2)
    if len(player2Possibilites) > 0:
        state["cur_player"] = player2
        state["possibilities"] = player2Possibilities
        return state
    
    #if player2 has no plays then player1 has to play again
    playerPossiblities = findAllPossibilities(state["board"], player)
    if len(playerPossiblities) > 0:
        state["possibilities"] = playerPossiblities
        return state
    
    #if no player can play -> game has ended
    coins, number = np.unique(state["board"], return_counts=True)
    final_count = dict(zip(coins, number))
    coins1 = final_count[1]
    conis2 = final_count[2]
    if coins1 < coins2: #black wins (2)
        state["winner"] = 2
    elif coins2 < coins1: #white wins (1)
        state["winner"] = 1
    else:                 # draw
        state["winner"] = 3
    
    

In [47]:
board = createBoard(8)
printBoard(board)

0	0 0 0 0 0 0 0 0 
1	0 0 0 0 0 0 0 0 
2	0 0 0 0 0 0 0 0 
3	0 0 0 1 2 0 0 0 
4	0 0 0 2 1 0 0 0 
5	0 0 0 0 0 0 0 0 
6	0 0 0 0 0 0 0 0 
7	0 0 0 0 0 0 0 0 

	0 1 2 3 4 5 6 7 


In [48]:
makeMove(board, (2, 4), 1)
printBoard(board)

0	0 0 0 0 0 0 0 0 
1	0 0 0 0 0 0 0 0 
2	0 0 0 0 1 0 0 0 
3	0 0 0 1 1 0 0 0 
4	0 0 0 2 1 0 0 0 
5	0 0 0 0 0 0 0 0 
6	0 0 0 0 0 0 0 0 
7	0 0 0 0 0 0 0 0 

	0 1 2 3 4 5 6 7 
